<div align="center">
  <a target="_blank" href="https://colab.research.google.com/github/crowdcent/crowdcent-challenge/blob/main/docs/tutorials/hyperliquid-example.ipynb">
    <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
  </a>
</div>

## Build a model on CrowdCent's training data and submit

### Load API key

For this tutorial, you will need:
1) **CrowdCent API Key**: register for free at https://crowdcent.com/accounts/signup/

In [1]:
%load_ext dotenv
%dotenv

import os
import crowdcent_challenge as cc
import polars as pl
from xgboost import XGBRegressor

CROWDCENT_API_KEY = os.environ["CROWDCENT_API_KEY"]

### Initialize the client

In [2]:
client = cc.ChallengeClient(
    challenge_slug="hyperliquid-ranking",
    api_key=CROWDCENT_API_KEY,
    base_url="http://localhost:8080/api",
)

2025-05-30 20:02:38,836 - INFO - ChallengeClient initialized for 'hyperliquid-ranking' at URL: http://localhost:8080/api


### Get CrowdCent's training data

In [ ]:
client.download_training_data(
    version="latest", dest_path="training_data.parquet"
)

training_data = pl.read_parquet("training_data.parquet")
training_data.head()

### Train a model on the training data

In [ ]:
xgb_regressor = XGBRegressor()
feature_names = [col for col in training_data.columns if col.startswith("feature")]

xgb_regressor.fit(
    training_data[feature_names],
    training_data[["target_10d", "target_30d"]],
)

### Get CrowdCent's latest inference data

In [3]:
# inference tickers
client.download_inference_data(
    release_date="current", dest_path="inference_data.parquet"
)

inference_data = pl.read_parquet("inference_data.parquet")
inference_data.head()

2025-05-30 20:02:45,517 - INFO - Downloading inference data for challenge 'hyperliquid-ranking' current to inference_data.parquet
2025-05-30 20:02:45,560 - INFO - Successfully downloaded inference data to inference_data.parquet


id,eodhd_id
str,str
"""BABY""","""BABY32198-USD.CC"""
"""OM""","""OM-USD.CC"""
"""IOTA""","""IOTA-USD.CC"""
"""MOODENG""","""MOODENG33093-USD.CC"""
"""ENS""","""ENS-USD.CC"""


### Make predictions on the inference data

In [ ]:
preds = xgb_regressor.predict(inference_data[feature_names])
preds.head()

### Submit to the `hyperliquid-ranking` challenge on CrowdCent

In [ ]:
client.submit_predictions(preds)